In [ ]:
import requests
import urllib3
import zipfile

urllib3.disable_warnings()
# This simply imports the requests package and the urllib3 package. Then I disable the warnings given as the URL does not have certificate verification enabled

# This downloads the .las file from the server. I have added AAA to the begining of the URL to make sure I don't get myself ban from PRISM
r = requests.get('AAAhttp://services.nacse.org/prism/data/public/normals/4km/ppt/14', auth=('user','pass'), verify = False)

open('PRISM_ppt_30yr_normal_4kmM2_all_bil.zip', 'wb').write(r.content)

with zipfile.ZipFile('PRISM_ppt_30yr_normal_4kmM2_all_bil.zip', 'r') as zip_ref:
    zip_ref.extractall(r'C:\Users\Alexander\Desktop\GIS Work\GIS5571\Lab1\GIS 5571 Lab2\PRISM')  # "r" is used to pass as raw string


In [1]:
# This creates an empty Mosaic Dataset called "PrismMosaic" that we will add our rasters to later
with arcpy.EnvManager(scratchWorkspace=r"C:\Users\Alexander\Desktop\GIS Work\GIS5571\Lab2\GIS 5571 Lab2\GIS 5571 Lab2.gdb", workspace=r"C:\Users\Alexander\Desktop\GIS Work\GIS5571\Lab2\GIS 5571 Lab2\GIS 5571 Lab2.gdb"):
    arcpy.management.CreateMosaicDataset(r"C:\Users\Alexander\Desktop\GIS Work\GIS5571\Lab2\GIS 5571 Lab2\GIS 5571 Lab2.gdb", "PrismMosaic", 'GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]', None, '', "NONE", None)

# This adds the rasters to the mosaic we just created
arcpy.management.AddRastersToMosaicDataset("PrismMosaic", "Raster Dataset", r"'C:\Users\Alexander\Desktop\GIS Work\PRISM'", "UPDATE_CELL_SIZES", "UPDATE_BOUNDARY", "NO_OVERVIEWS", None, 0, 1500, None, '', "SUBFOLDERS", "ALLOW_DUPLICATES", "NO_PYRAMIDS", "NO_STATISTICS", "NO_THUMBNAILS", '', "NO_FORCE_SPATIAL_REFERENCE", "NO_STATISTICS", None, "NO_PIXEL_CACHE", r"C:\Users\Alexander\AppData\Local\ESRI\rasterproxies\PrismMosaic")

#This adds the ppm variable field to the footprint section of the PrismMosaic we created
arcpy.management.CalculateField(r"PrismMosaic\Footprint", "Variable", '"ppt"', "PYTHON3", '', "TEXT", "NO_ENFORCE_DOMAINS")

# This adds the timestamp date field to the footprint section of the PrismMosaic we created
arcpy.management.CalculateField(r"PrismMosaic\Footprint", "Timestamp", "DateAdd(Date(1981,0,1),1$feature.OBJECTID-1,'month')", "ARCADE", '', "DATE", "NO_ENFORCE_DOMAINS")

# This adds the time dimension to the mosaic dataset
with arcpy.EnvManager(scratchWorkspace=r"C:\Users\Alexander\Desktop\GIS Work\GIS5571\Lab2\GIS 5571 Lab2\GIS 5571 Lab2.gdb", workspace=r"C:\Users\Alexander\Desktop\GIS Work\GIS5571\Lab2\GIS 5571 Lab2\GIS 5571 Lab2.gdb"):
    arcpy.md.BuildMultidimensionalInfo("PrismMosaic", "Variable", "Timestamp # #", None, "NO_DELETE_MULTIDIMENSIONAL_INFO")

# This converts the mosaic into a single Multidimensional Raster Layer
arcpy.md.MakeMultidimensionalRasterLayer("PrismMosaic", "PrismMosaic_MultidimLayer", "ppt", "ALL", None, None, '', '', '', None, '', "-125.020833333333 24.0625 -66.479166666662 49.937500000002", "DIMENSIONS")

# This creates the Space Time Cube
arcpy.stpm.CreateSpaceTimeCubeMDRasterLayer("PrismMosaic_MultidimLayer", r"C:\Users\Alexander\Desktop\GIS Work\GIS5571\Lab2\GIS 5571 Lab2\PrismCube.nc", "ZEROS")

